In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 10

## 11.1. 날짜, 시간자료형, 도구  
datetime,time,calande모듈

In [2]:
from datetime import datetime
now = datetime.now()
now
now.year,now.month,now.day

datetime.datetime(2019, 9, 1, 16, 15, 52, 370693)

(2019, 9, 1)

datetime은 날짜와 시간을 모두 저장하며, 마이크로초까지 지원함

In [3]:
now.second;now.microsecond
now.microsecond//1000;now.microsecond%1000

52

370693

370

693

datetime.timedelta는 두 객체간 시간적 차이를 표현가능

In [4]:
delta = datetime(2011,1,7) - datetime(2008,6,24,8,15)
delta
delta.days
delta.seconds

datetime.timedelta(days=926, seconds=56700)

926

56700

time delta를 더하거나 빼면 그만큼의 시간이 datetime객체에 적용되어 새 객체를 만듦

In [5]:
from datetime import timedelta
start = datetime(2011,1,7)
start
start + timedelta(12)#일,초,마이크로초
start - 2* timedelta(2)

datetime.datetime(2011, 1, 7, 0, 0)

datetime.datetime(2011, 1, 19, 0, 0)

datetime.datetime(2011, 1, 3, 0, 0)

### 11.1.1 문자열을 datetime으로 변환하기  
포맷규칙을 넘겨서 문자열로 나타낼 수 있음

In [6]:
stamp = datetime(2011,1,3)
stamp
str(stamp)
stamp.strftime('%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

'2011-01-03 00:00:00'

'2011-01-03'

In [7]:
#문자열을 날짜로 변환할 수도 있음
value = '2011-01-03'
datetime.strptime(value,'%Y-%m-%d')

datestrs = ['7/6/2011','8/6/2011']
[datetime.strptime(value,'%d/%m/%Y') for value in datestrs]

datetime.datetime(2011, 1, 3, 0, 0)

[datetime.datetime(2011, 6, 7, 0, 0), datetime.datetime(2011, 6, 8, 0, 0)]

datetime.strptime을 사용하는것이 강력하긴 하지만 매번 규칙을 명시하는것은 귀찮음..  
dateutil에 parse.parse메서드를 쓰면 편리함 (Pandas와 함께 설치됨)

In [8]:
from dateutil.parser import parse

parse('2011-06-07')
parse('jAn 31,2019 11:40')
'''사람이 인지하는 거의 대부분의 표현방식을 파싱가능'''

datetime.datetime(2011, 6, 7, 0, 0)

datetime.datetime(2019, 1, 31, 11, 40)

'사람이 인지하는 거의 대부분의 표현방식을 파싱가능'

In [9]:
#날짜가 월 앞에 오는경우 dayfirst=True
parse('6/12/2011',dayfirst = True)

datetime.datetime(2011, 12, 6, 0, 0)

In [10]:
#to_datetime
datestrs = ['2011-07-06 12:00:00','2011-09-06 12:00:00']
pd.to_datetime(datestrs)

#누락된값으로 간주해야할 값도 처리해줌
idx = pd.to_datetime(datestrs + [None])
idx
pd.isnull(idx) #Not a Time

DatetimeIndex(['2011-07-06 12:00:00', '2011-09-06 12:00:00'], dtype='datetime64[ns]', freq=None)

DatetimeIndex(['2011-07-06 12:00:00', '2011-09-06 12:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

array([False, False,  True])

## 11.2. 시계열 기초  
가장 기본적인 시계열 객체는 문자열이나 datetime객체로 표현되는 타임스탬프 Series

In [11]:
from datetime import datetime
dates = [datetime(2011,1,2),datetime(2011,1,5),
        datetime(2011,1,7),datetime(2011,1,8),
        datetime(2011,1,10),datetime(2011,1,12)]
ts = pd.Series(np.random.randn(6),index = dates)
ts
ts.index

2011-01-02   -0.995204
2011-01-05    0.240292
2011-01-07    1.071618
2011-01-08   -1.163677
2011-01-10   -1.276086
2011-01-12    1.022352
dtype: float64

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

numpy의 datetime64를 사용해 나노초 단위로 타임스탬프 저장  
DatetimeIndex의 스칼라값은 Timestamp 객체

In [12]:
ts.index.dtype
ts.index[0]

dtype('<M8[ns]')

Timestamp('2011-01-02 00:00:00')

In [13]:
ts + ts[::2]

2011-01-02   -1.990409
2011-01-05         NaN
2011-01-07    2.143236
2011-01-08         NaN
2011-01-10   -2.552172
2011-01-12         NaN
dtype: float64

### 11.2.1 색인, 선택, 부분선택

시계열의 인덱싱은 시리즈와 동일함

In [14]:
ts
stamp = ts.index[2]
ts[stamp]

2011-01-02   -0.995204
2011-01-05    0.240292
2011-01-07    1.071618
2011-01-08   -1.163677
2011-01-10   -1.276086
2011-01-12    1.022352
dtype: float64

1.071617970810501

해석가능한 문자를 넘겨서 편리하게 사용할 수 있음

In [15]:
ts['1/7/2011']
ts['20110107']

1.071617970810501

1.071617970810501

긴 시계열에서 연이나 월만 넘겨서 데이터 일부를 선택할수도있음

In [16]:
longer_ts = pd.Series(np.random.randn(1000),
                     index = pd.date_range('1/1/2000',periods=1000))
longer_ts
longer_ts['2001']

2000-01-01    0.517923
2000-01-02   -0.233374
2000-01-03    0.540131
2000-01-04    1.560119
2000-01-05   -1.341512
                ...   
2002-09-22   -0.397576
2002-09-23    2.573913
2002-09-24   -0.226919
2002-09-25   -0.352197
2002-09-26    0.799597
Freq: D, Length: 1000, dtype: float64

2001-01-01   -2.237479
2001-01-02    0.901510
2001-01-03   -0.360812
2001-01-04   -1.668400
2001-01-05   -0.034584
                ...   
2001-12-27    0.410207
2001-12-28    0.465194
2001-12-29    1.730561
2001-12-30    0.827635
2001-12-31    1.827207
Freq: D, Length: 365, dtype: float64

In [17]:
longer_ts['2001-05']

2001-05-01    1.271955
2001-05-02   -1.628566
2001-05-03   -0.424852
2001-05-04   -0.682238
2001-05-05    2.111846
                ...   
2001-05-27    0.141427
2001-05-28    0.506224
2001-05-29   -1.314526
2001-05-30    0.449695
2001-05-31    0.404674
Freq: D, Length: 31, dtype: float64

슬라이싱도 유사함

In [18]:
ts[datetime(2011,1,7):]

2011-01-07    1.071618
2011-01-08   -1.163677
2011-01-10   -1.276086
2011-01-12    1.022352
dtype: float64

In [19]:
ts
ts['1/6/2011':'01/11/2011']

2011-01-02   -0.995204
2011-01-05    0.240292
2011-01-07    1.071618
2011-01-08   -1.163677
2011-01-10   -1.276086
2011-01-12    1.022352
dtype: float64

2011-01-07    1.071618
2011-01-08   -1.163677
2011-01-10   -1.276086
dtype: float64

슬라이스된 데이터는 원본의 뷰이므로 변경사항이 원본에 적용된다는것에 유의!

truncate 인스턴스 메서드는 TimeSeries를 두개의 날짜로 나눔

In [20]:
ts
ts.truncate(before='1/9/2011')
ts.truncate(after='1/9/2011')

2011-01-02   -0.995204
2011-01-05    0.240292
2011-01-07    1.071618
2011-01-08   -1.163677
2011-01-10   -1.276086
2011-01-12    1.022352
dtype: float64

2011-01-10   -1.276086
2011-01-12    1.022352
dtype: float64

2011-01-02   -0.995204
2011-01-05    0.240292
2011-01-07    1.071618
2011-01-08   -1.163677
dtype: float64

**DataFrame에도 동일하게 적용되며, 로우에 인덱싱됨**

In [21]:
dates = pd.date_range('1/1/2000',periods = 100,freq='W-WED')
long_df = pd.DataFrame(np.random.randn(100,4),
                      index = dates,
                      columns = ['Colorado','Texas','New York','Ohio'])
long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,0.544454,-0.508861,1.108997,0.026263
2001-05-09,0.417397,-0.432595,0.315008,-0.968983
2001-05-16,0.517590,2.890043,-0.176081,-1.697976
2001-05-23,1.216835,0.174666,-0.317490,-1.120706
2001-05-30,0.333685,0.250676,0.016158,0.022365


### 11.2.2. 중복된 색인을 갖는 시계열

In [22]:
dates = pd.DatetimeIndex(['1/1/2000','1/2/2000','1/2/2000','1/2/2000','1/3/2000'])
dup_ts = pd.Series(np.arange(5),index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int32

is_unique 속성을 보면 색인이 유일하지 않음을 확인가능

In [23]:
dup_ts.index.duplicated()

array([False, False,  True,  True, False])

In [24]:
dup_ts.index.is_unique

False

이걸 인덱싱하면 중복여부에 따라 스칼라 혹은 슬라이스가 생성

In [25]:
dup_ts['1/3/2000']
dup_ts['1/2/2000']

4

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int32

유일하지 않은 타임스탬프를 가진 데이터를 집계한다고 하면, 한가지 방법은 groupby에 level=0(단일단계 인덱싱)을 넘기는것

In [26]:
# dup_ts.groupby(dup_ts.index).mean()
dup_ts.groupby(level=0).mean()
dup_ts.groupby(level=0).count()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int32

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

## 11.3. 날짜 범위, 빈도, 이동  

In [27]:
#시계열을 고정된 일 빈도로 변환하기 (resample)
ts
resampler = ts.resample('D')
resampler

2011-01-02   -0.995204
2011-01-05    0.240292
2011-01-07    1.071618
2011-01-08   -1.163677
2011-01-10   -1.276086
2011-01-12    1.022352
dtype: float64

DatetimeIndexResampler [freq=<Day>, axis=0, closed=left, label=left, convention=start, base=0]

### 11.3.1. 날짜 범위 생성하기  
pandas.date_range를 사용하면 특정 빈도에 따라 지정한 길이만큼의 DatetimeIndex를 생성가능

In [28]:
index = pd.date_range('2012-04-01','2012-06-01')
index
'''기본적으로 일별 타임스탬프 생성'''

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

'기본적으로 일별 타임스탬프 생성'

In [29]:
#시작날짜나 종료날짜만 넘기다면 생성할 기간의 숫자를 함께 전달해야함
pd.date_range(start = '2012-04-01',periods=20)
pd.date_range(end = '2012-04-01',periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['2012-03-13', '2012-03-14', '2012-03-15', '2012-03-16',
               '2012-03-17', '2012-03-18', '2012-03-19', '2012-03-20',
               '2012-03-21', '2012-03-22', '2012-03-23', '2012-03-24',
               '2012-03-25', '2012-03-26', '2012-03-27', '2012-03-28',
               '2012-03-29', '2012-03-30', '2012-03-31', '2012-04-01'],
              dtype='datetime64[ns]', freq='D')

In [30]:
pd.date_range('2000-01-01','2000-12-01',freq='BM') #해당 기간내,영업마감일만 표시하기

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

In [31]:
#시작이나 종료시간의 타임스탬프를 보존함
pd.date_range('2000-01-01 12:00:24',periods=10)
pd.date_range(end='2000-01-01 12:00:24',periods=10)
pd.date_range('2000-01-01 12:00:24','2000-01-05 19:20:24')

DatetimeIndex(['2000-01-01 12:00:24', '2000-01-02 12:00:24',
               '2000-01-03 12:00:24', '2000-01-04 12:00:24',
               '2000-01-05 12:00:24', '2000-01-06 12:00:24',
               '2000-01-07 12:00:24', '2000-01-08 12:00:24',
               '2000-01-09 12:00:24', '2000-01-10 12:00:24'],
              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['1999-12-23 12:00:24', '1999-12-24 12:00:24',
               '1999-12-25 12:00:24', '1999-12-26 12:00:24',
               '1999-12-27 12:00:24', '1999-12-28 12:00:24',
               '1999-12-29 12:00:24', '1999-12-30 12:00:24',
               '1999-12-31 12:00:24', '2000-01-01 12:00:24'],
              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['2000-01-01 12:00:24', '2000-01-02 12:00:24',
               '2000-01-03 12:00:24', '2000-01-04 12:00:24',
               '2000-01-05 12:00:24'],
              dtype='datetime64[ns]', freq='D')

시간정보도 포함하여 갖고있지만, 관례에 맞추어 자정에 맞추어 타임스탬프를 정규화 하고 싶다면 normalize 옵션 사용

In [32]:
pd.date_range('2012-05-02 12:51:31',periods=5,normalize = True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

### 11.3.2. 빈도와 날짜 옵셋  
pandas에서 빈도는 기본빈도와 배수의 조합으로 이루어짐  
각각의 기본빈도는 날짜오프셋이라는 객체를 사용가능

In [33]:
from pandas.tseries.offsets import Hour,Minute
hour=  Hour()
hour

<Hour>

In [34]:
four_hours = Hour(4)
four_hours

<4 * Hours>

대부분의 어플리케이션에서는 위처럼 직접 객체를 만들 경우는 없고 'H','4H'처럼 문자열로 표현

In [35]:
pd.date_range('2001-01-01','2001-01-03',freq='4H')

DatetimeIndex(['2001-01-01 00:00:00', '2001-01-01 04:00:00',
               '2001-01-01 08:00:00', '2001-01-01 12:00:00',
               '2001-01-01 16:00:00', '2001-01-01 20:00:00',
               '2001-01-02 00:00:00', '2001-01-02 04:00:00',
               '2001-01-02 08:00:00', '2001-01-02 12:00:00',
               '2001-01-02 16:00:00', '2001-01-02 20:00:00',
               '2001-01-03 00:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [36]:
#여러 옵셋을 덧셈으로 합치기
Hour(2)+Minute(30)

#문자열로 넘겨도 같은 표현으로 해석됨
pd.date_range('2001-01-01','2001-01-02',freq='1h30min')

<150 * Minutes>

DatetimeIndex(['2001-01-01 00:00:00', '2001-01-01 01:30:00',
               '2001-01-01 03:00:00', '2001-01-01 04:30:00',
               '2001-01-01 06:00:00', '2001-01-01 07:30:00',
               '2001-01-01 09:00:00', '2001-01-01 10:30:00',
               '2001-01-01 12:00:00', '2001-01-01 13:30:00',
               '2001-01-01 15:00:00', '2001-01-01 16:30:00',
               '2001-01-01 18:00:00', '2001-01-01 19:30:00',
               '2001-01-01 21:00:00', '2001-01-01 22:30:00',
               '2001-01-02 00:00:00'],
              dtype='datetime64[ns]', freq='90T')

월 마지막일 이나 월 영업 마감일은 월말이 주말인지 아닌지에 따라 다르므로 따로 앵커드 옵셋이라 부르겠음

**월별 주차(WOM)**  
이걸 이용하면 매월 셋째주 금요일 등을 얻을수 있음

In [37]:
rng = pd.date_range('2012-01-01','2012-09-01',freq='WOM-3FRI')
rng

DatetimeIndex(['2012-01-20', '2012-02-17', '2012-03-16', '2012-04-20',
               '2012-05-18', '2012-06-15', '2012-07-20', '2012-08-17'],
              dtype='datetime64[ns]', freq='WOM-3FRI')

### 11.3.3. 데이터 시프트  
데이터를 시간축에서 앞이나 뒤로 이동하는것(느슨한 시프트)

In [38]:
ts = pd.Series(np.random.randn(4),index=pd.date_range('1/1/2000',periods=4,freq='M'))
ts
ts.shift(2)
ts.shift(-2)

2000-01-31    0.583637
2000-02-29    0.421425
2000-03-31   -0.911000
2000-04-30   -1.496993
Freq: M, dtype: float64

2000-01-31         NaN
2000-02-29         NaN
2000-03-31    0.583637
2000-04-30    0.421425
Freq: M, dtype: float64

2000-01-31   -0.911000
2000-02-29   -1.496993
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

시프트는 한시계열 내에서 퍼센트 변화를 계산할때 흔히 사용함

In [39]:
ts/ts.shift(1)-1

2000-01-31         NaN
2000-02-29   -0.277932
2000-03-31   -3.161711
2000-04-30    0.643242
Freq: M, dtype: float64

느슨한 시프트는 색인을 안바꾸니까 어떤 데이터는 버려짐,  
빈도를 알고있다면 shift에 빈도를 넣어서 타임스태프를 확장할수있음

In [40]:
ts
ts.shift(2)
ts.shift(2,freq='M')

2000-01-31    0.583637
2000-02-29    0.421425
2000-03-31   -0.911000
2000-04-30   -1.496993
Freq: M, dtype: float64

2000-01-31         NaN
2000-02-29         NaN
2000-03-31    0.583637
2000-04-30    0.421425
Freq: M, dtype: float64

2000-03-31    0.583637
2000-04-30    0.421425
2000-05-31   -0.911000
2000-06-30   -1.496993
Freq: M, dtype: float64

다른 빈도를 사용해도 되므로 유연하게 데이터를 밀고 당길 수 있음

In [41]:
ts
ts.shift(3,freq='D')
ts.shift(1,freq='90T') #T는 분

2000-01-31    0.583637
2000-02-29    0.421425
2000-03-31   -0.911000
2000-04-30   -1.496993
Freq: M, dtype: float64

2000-02-03    0.583637
2000-03-03    0.421425
2000-04-03   -0.911000
2000-05-03   -1.496993
dtype: float64

2000-01-31 01:30:00    0.583637
2000-02-29 01:30:00    0.421425
2000-03-31 01:30:00   -0.911000
2000-04-30 01:30:00   -1.496993
Freq: M, dtype: float64

**오프셋만큼 날짜 시프트하기**